# pyUCell - some important parameters

This document describes some important parameters of the pyUCell algorithm, and how they can be adapted depending on your dataset.

For a simple pyUCell tutorial refer to: [pyUCell basics](https://pyucell.readthedocs.io/en/latest/notebooks/basic.html#)

## Load example dataset

In [14]:
import scanpy as sc
import pyucell as uc

In [15]:
adata = sc.datasets.pbmc3k()

**Note:** becase UCell scores are based on relative gene ranks, it can be applied both on raw counts or normalized data. As long as the normalization preserves the relative ranks between genes, the results will be equivalent.

## 1. Positive and negative gene sets in signatures

pyUCell supports **positive and negative gene sets** within a signature. Simply append + or - signs to the genes to include them in positive and negative sets, respectively. For example:

In [16]:
signatures = {"CD8T": ["CD8A+", "CD8B+", "CD4-"], "CD4T": ["CD4+", "CD40LG+", "CD8A-", "CD8B-"]}

pyUCell evaluates the positive and negative gene sets separately, then subtracts the scores. The parameter `w_neg` controls the relative weight of the negative gene set compared to the positive set (`w_neg=1.0` means equal weight). Note that the combined score is clipped to zero, to preserve UCell scores in the [0, 1] range.

In [17]:
uc.compute_ucell_scores(adata, signatures=signatures, w_neg=1)

In [18]:
adata.obs

,CD8T_UCell,CD4T_UCell
index,,
AAACATACAACCAC-1,0.770771,0.000000
AAACATTGAGCTAC-1,0.000000,0.000000
AAACATTGATCAGC-1,0.000000,0.252586
AAACCGTGCTTCCG-1,0.000000,0.000000
AAACCGTGTATGCG-1,0.000000,0.000000
...,...,...
TTTCGAACTCTCAT-1,0.000000,0.000000
TTTCTACTGAGGCA-1,0.000000,0.000000
TTTCTACTTCCTCG-1,0.000000,0.000000


## 2. The `max_rank` parameter

Single-cell data are sparse. In other words, for any given cell only a few hundred/a few thousand genes (out of tens of thousands) are detected with at least one UMI count. Because UCell scores are based on ranking genes by their expression values, it is essential to account for data sparsity when calculating ranks. This is implemented by capping ranks to a `max_rank` parameter, in other words only the top `max_rank` genes are ranked, and the rest are assumed equivalent at the lowest ranking value.

It is often useful to adjust the `max_rank` depending on the sparsity of your dataset. A good rule of thumb is to examine the median number of expressed genes per cell, and set `max_rank` in that order of magnitude. For example, for the test dataset:

In [27]:
aa = adata.copy()
sc.pp.calculate_qc_metrics(aa, inplace=True)

sc.pl.violin(aa, "n_genes_by_counts", jitter=0.4, size=1, log=True, groupby=None)

AttributeError: 'function' object has no attribute 'X'

This dataset has relatively low depth, so it is advisable to choose a `max_rank` around `800-1000` (from the default `1500`)

In [20]:
uc.compute_ucell_scores(adata, signatures=signatures, max_rank=800)

This is even more important when applying pyUCell to technologies/modalities of much lower dimensionality, for example probe-based spatial transcriptomics data (e.g. Xenium, CosMx), or antibody tags (ADT) in CITE-seq experiments. Xenium panels contain a few hundred/a few thousand genes; CITE-seq can detect a few hundred proteins, as opposed to thousands of genes in scRNA-seq. The `max_rank` parameter should then also be adjusted to reflect the new dimensionality and set to no more than the number of probes in the panel.

## 3. Handling missing genes

If a subset of the genes in your signature are absent from the count matrix, how should they be handled?

pyUCell offers two alternative ways of handling missing genes:
- `missing_genes="impute"` (default): it assumes that absence from the count matrix means zero expression. All values for this gene are imputed to zero.
-  `missing_genes="skip"`: simply exclude all missing genes from the signatures; they won't contribute to the scores.

In [21]:
signatures = {"CD8T": ["CD8A+", "CD8B+", "CD4-", "notagene"]}
uc.compute_ucell_scores(adata, signatures=signatures, missing_genes="impute")
adata.obs

,CD8T_UCell,CD4T_UCell,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_50_genes,pct_counts_in_top_100_genes,pct_counts_in_top_200_genes,pct_counts_in_top_500_genes
index,,,,,,,,,,
AAACATACAACCAC-1,0.514019,0.000000,781,6.661855,2421.0,7.792349,47.748864,63.279637,74.969021,88.393226
AAACATTGAGCTAC-1,0.000000,0.000000,1352,7.210080,4903.0,8.497807,45.502753,61.023863,71.813176,82.622884
AAACATTGATCAGC-1,0.000000,0.035692,1131,7.031741,3149.0,8.055158,41.314703,53.794856,65.449349,79.961893
AAACCGTGCTTCCG-1,0.000000,0.000000,960,6.867974,2639.0,7.878534,39.029936,52.898825,66.691929,82.569155
AAACCGTGTATGCG-1,0.000000,0.000000,522,6.259581,981.0,6.889591,44.852192,55.657492,67.176351,97.757390
...,...,...,...,...,...,...,...,...,...,...
TTTCGAACTCTCAT-1,0.000000,0.000000,1155,7.052721,3461.0,8.149602,39.237215,52.528171,65.761341,81.074834
TTTCTACTGAGGCA-1,0.000000,0.000000,1227,7.113142,3447.0,8.145550,37.278793,51.900203,63.881636,78.909196
TTTCTACTTCCTCG-1,0.000000,0.000000,622,6.434547,1684.0,7.429521,45.783848,61.638955,74.940618,92.755344


In [22]:
uc.compute_ucell_scores(adata, signatures=signatures, missing_genes="skip")
adata.obs

,CD8T_UCell,CD4T_UCell,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_50_genes,pct_counts_in_top_100_genes,pct_counts_in_top_200_genes,pct_counts_in_top_500_genes
index,,,,,,,,,,
AAACATACAACCAC-1,0.770771,0.000000,781,6.661855,2421.0,7.792349,47.748864,63.279637,74.969021,88.393226
AAACATTGAGCTAC-1,0.000000,0.000000,1352,7.210080,4903.0,8.497807,45.502753,61.023863,71.813176,82.622884
AAACATTGATCAGC-1,0.000000,0.035692,1131,7.031741,3149.0,8.055158,41.314703,53.794856,65.449349,79.961893
AAACCGTGCTTCCG-1,0.000000,0.000000,960,6.867974,2639.0,7.878534,39.029936,52.898825,66.691929,82.569155
AAACCGTGTATGCG-1,0.000000,0.000000,522,6.259581,981.0,6.889591,44.852192,55.657492,67.176351,97.757390
...,...,...,...,...,...,...,...,...,...,...
TTTCGAACTCTCAT-1,0.000000,0.000000,1155,7.052721,3461.0,8.149602,39.237215,52.528171,65.761341,81.074834
TTTCTACTGAGGCA-1,0.000000,0.000000,1227,7.113142,3447.0,8.145550,37.278793,51.900203,63.881636,78.909196
TTTCTACTTCCTCG-1,0.000000,0.000000,622,6.434547,1684.0,7.429521,45.783848,61.638955,74.940618,92.755344


## 4. Chunk size

UCell scores are calculated individually for each cell (though they may be later [smoothed](https://pyucell.readthedocs.io/en/latest/notebooks/knn.html) by nearest-neighbor similarity). This means that computation can be easily split into batches, reducing the computational footprint of gene ranking and enabling parallel processing (see below). The size of the batches is controlled by the `chunk_size` parameter. Large chunks take up more RAM, while small chunk sizes have large overhead from dataset splitting and merging. A sweet spot for `chunk_size` is usually in the order of 100-1000 cells per batch.

In [23]:
uc.compute_ucell_scores(adata, signatures=signatures, chunk_size=100)

## 5. Parallelization

Parallelization is handled internally by `joblib` and the `Parallel` module. You may control the number of jobs with the `n_jobs` parameter. By default all available cores are used (`n_jobs=-1`).

In [24]:
%time uc.compute_ucell_scores(adata, signatures=signatures, n_jobs=4)

CPU times: user 52.4 ms, sys: 55.8 ms, total: 108 ms
Wall time: 3.74 s


In [25]:
%time uc.compute_ucell_scores(adata, signatures=signatures, n_jobs=1)

CPU times: user 754 ms, sys: 24.6 ms, total: 779 ms
Wall time: 778 ms


## References

If you used UCell in your research, please cite:

**UCell: robust and scalable single-cell gene signature scoring.** Massimo Andreatta & Santiago J Carmona (2021) 
CSBJ - [doi.org/10.1016/j.csbj.2021.06.043](https://doi.org/10.1016/j.csbj.2021.06.043)